In [1]:
import nltk
import numpy as np
import re
from nltk.corpus import cmudict
#http://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.cmudict

In [2]:
ARPAbet = ['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']
arpa = {}
for i in range(len(ARPAbet)):
    arpa[ARPAbet[i]] = i
print len(ARPAbet)

39


In [5]:
d = cmudict.dict()
e = cmudict.entries()
x_train = []
y_train = []
for x in e:
    #print x
    w = x[0]
    skip = False
    for c in w:
        if ord(c) < ord('a') or ord(c) > ord('z'):
            skip = True
    if skip:
        continue
    w.encode('ascii','ignore')
    w = list(w)
    x_train.append(w)
    p = []
    for y in x[1]:
        stress = -1
        y.encode('ascii','ignore')
        if(y[0:-1] not in arpa):
            continue
        stress = arpa[y[0:-1]] * 2
        if(y[len(y) - 1] == '1'):
            stress += 1
        p.append(stress)
    y_train.append(p)
maxlen = 35
X = np.zeros((len(x_train), maxlen, 26), dtype=np.bool)
y = np.zeros((len(y_train), maxlen, len(ARPAbet) * 2 + 1), dtype=np.bool)
for i, word in enumerate(x_train):
    for t, char in enumerate(word):
        #print word
        X[i, t, ord(char) - ord('a')] = 1
    
    for j, s in enumerate(y_train[i]):
        y[i, j, s] = 1


In [10]:
print d["goodbye"]
print e[110]
print x_train[11101]
c = 0
for x in e:
    c = max(c, len(x[0]))
print c
print y[0]

[[u'G', u'UH2', u'D', u'B', u'AY1']]
(u'abducted', [u'AH0', u'B', u'D', u'AH1', u'K', u'T', u'IH0', u'D'])
[u'b', u'l', u'a', u'c', u'k', u'o', u'u', u't', u's']
33
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]


In [19]:
# Build the model.
from keras.models import Sequential
from keras.layers.core import TimeDistributedDense, Dropout, Activation, Dense, RepeatVector
from keras.layers.recurrent import LSTM
from keras.optimizers import RMSprop, adam
model = Sequential()
model.add(LSTM(300, input_shape=(maxlen, 26)))
model.add(RepeatVector(35))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributedDense(len(ARPAbet) * 2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam(), metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
lstm_10 (LSTM)                     (None, 300)         392400      lstm_input_7[0][0]               
____________________________________________________________________________________________________
repeatvector_4 (RepeatVector)      (None, 35, 300)     0           lstm_10[0][0]                    
____________________________________________________________________________________________________
lstm_11 (LSTM)                     (None, 35, 100)     160400      repeatvector_4[0][0]             
____________________________________________________________________________________________________
timedistributeddense_3 (TimeDistrib(None, 35, 78)      7878        lstm_11[0][0]                    
Total params: 560678
______________________________________________________________________

In [20]:

#model.load_weights('model.h5')
model.fit(X, y, batch_size=100, nb_epoch=15)
model.save_weights('many.h5', overwrite=True)

Epoch 1/15
124996/124996 [==============================] - 287s - loss: 0.2180 - acc: 0.0145   
Epoch 2/15
124996/124996 [==============================] - 298s - loss: 0.2163 - acc: 0.0146   
Epoch 3/15
124996/124996 [==============================] - 297s - loss: 0.2163 - acc: 0.0145   
Epoch 4/15
124996/124996 [==============================] - 287s - loss: 0.2162 - acc: 0.0145   
Epoch 5/15
124996/124996 [==============================] - 286s - loss: 0.2162 - acc: 0.0145   
Epoch 6/15
124996/124996 [==============================] - 285s - loss: 0.2162 - acc: 0.0146   
Epoch 7/15
124996/124996 [==============================] - 292s - loss: 0.2162 - acc: 0.0145   
Epoch 8/15
124996/124996 [==============================] - 291s - loss: 0.2161 - acc: 0.0145   
Epoch 9/15
124996/124996 [==============================] - 279s - loss: 0.2161 - acc: 0.0146   
Epoch 10/15
124996/124996 [==============================] - 279s - loss: 0.2161 - acc: 0.0145   
Epoch 11/15
124996/124996 [==

In [26]:
def stress(w):
    w.encode('ascii','ignore')
    w = list(w)
    X = np.zeros((1, maxlen, 26), dtype=np.bool)
    for t, char in enumerate(w):
        X[0, t, ord(char) - ord('a')] = 1
    m = model.predict(X)
    #print X
    #a = np.argmax(m)
    #print a
    for t, c in enumerate(m[0]):
        a = np.argmax(c)
        if(a == 0)
    return m.shape
    return(ARPAbet[a])
print stress('l')
print 

(1L, 35L, 78L)



In [8]:
model.save_weights('finalweights.h5', overwrite=True)